In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import requests
import shutil
import bs4
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
import pyautogui
import cv2
import time
import re

In [2]:
#import excel that contains tiktok link
# df_link = pd.read_excel('D://Link Tiktok.xlsx')

df_link = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQQriTN2AS9ibYF8-8AmZY2R4QMM2CKj16XsxoueBq1OBpiGyeLGC5GVTaQwHSk0amBVCqyh1e0RHT6/pub?output=csv"
df_link = pd.read_csv(df_link)

In [3]:
#FUNTIONS

#download image function
def down(image, xpath):
    #get link image
    captcha = driver.find_element (By.XPATH, xpath)
    src = captcha.get_attribute('src')

    #download image
    response = requests.get(src, stream=True)
    with open(image, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response

#CONTOURS FUNCTION
def bin_img(image):
    img = cv2.imread(image)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    binary_img = cv2.threshold(img_gray, 242, 255, cv2.THRESH_BINARY_INV)[1]

    return binary_img

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.keys import Keys
import bs4
from PIL import Image
import pytesseract

links = []
title = []
username = []
likes = []
views = []
comment = []
share = []

#get url from dataframe
for i in range(len(df_link.iloc[:,0])):
    #open driver chrome
    PATH = "C://chromedriver.exe"
    driver = webdriver.Chrome(PATH)

    #get tiktok link
    link = df_link.iloc[i,0]

    #open and maximize window
    driver.get(link)
    max = driver.maximize_window()

    #wait until captcha window pop up
    try:
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//*[@id='captcha-verify-image']")))
        pass
    except TimeoutException:
        links.append(link)
        title.append('skip')
        username.append('skip')
        likes.append('skip')
        views.append('skip')
        comment.append('skip')
        share.append('skip')
        continue

    #click zoom in to 125x
    pyautogui.moveTo(1898, 57, duration=0.5)
    pyautogui.click()
    time.sleep(0.5)
    pyautogui.moveTo(1836, 238, duration=0.5)
    pyautogui.doubleClick()

    #CAPTCHA SOLVER
    #download captcha image
    down('captcha_img.jpeg', "//*[@id='captcha-verify-image']")
    #download piece of captcha image
    down('captcha_piece.jpeg', "//*[@id='tiktok-verify-ele']/div/div[2]/img[2]")

    #read img
    img = cv2.imread('captcha_img.jpeg')
    img_piece = cv2.imread('captcha_piece.jpeg')

    capt = bin_img('captcha_img.jpeg')
    piece = bin_img('captcha_piece.jpeg')

    #matching shape
    match = cv2.matchTemplate(capt, piece, cv2.TM_CCOEFF_NORMED)

    #check x, y in maximum value
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(match)

    #width and height
    w = piece.shape[1]
    h = piece.shape[0]

    time.sleep(2)

    #move mouse to slider button
    pyautogui.moveTo(782, 722, duration=1.5)
    #drag mouse to the right position
    pyautogui.dragRel((0.78*max_loc[0]), 0, button='left', duration=2)

    #wait until captcha pop up dissapear
    try:
        WebDriverWait(driver, 15).until_not(EC.visibility_of_element_located((By.XPATH, "//*[@id='captcha-verify-image']")))
        pass
    except TimeoutException:
        links.append(link)
        title.append('skip')
        username.append('skip')
        likes.append('skip')
        views.append('skip')
        comment.append('skip')
        share.append('skip')
        continue

    #exit login bar
    #exit login bar
    ext_login_bar = driver.find_element(By.XPATH, '//*[@id="login-modal"]/div[2]')
    ext_login_bar.click()

    time.sleep(1)

    #start and pause the video
    pyautogui.moveTo(367, 995, duration=1)
    pyautogui.click(button='left')
    time.sleep(2)
    pyautogui.click(button='left')

    time.sleep(1)

    #ADD USERNAME, LIKES, COMMENT, SHARE
    #find username
    xpath_username = '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[2]/div/a[2]/span[1]'
    try:
        link_username = driver.find_element(By.XPATH, xpath_username)
        get_link_username = link_username.text
    except:
        get_link_username = 'skip'
    #find likes
    xpath_likes = '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[1]/div[1]/div[4]/div/button[1]/strong'
    try:
        link_likes = driver.find_element(By.XPATH, xpath_likes)
        get_link_likes = link_likes.text
    except:
        get_link_likes = 'skip'
    #find comment
    xpath_comment = '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[1]/div[1]/div[4]/div/button[2]/strong'
    try:
        link_comment = driver.find_element(By.XPATH, xpath_comment)
        get_link_comment = link_comment.text
    except:
        get_link_comment = 'skip'
    #find share
    xpath_share = '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[1]/div[1]/div[4]/div/button[3]/strong'
    try:
        link_share = driver.find_element(By.XPATH, xpath_share)
        get_link_share = link_share.text
    except:
        get_link_share = 'skip'

    #adding info to list
    username.append(get_link_username)
    likes.append(get_link_likes)
    comment.append(get_link_comment)
    share.append(get_link_share)

    #open profile
    profile = driver.find_element(By.XPATH, '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[2]/div/a[2]/span[1]')
    profile.click()

    time.sleep(2)

    #scroll to the bottom
    previous_height = driver.execute_script('return document.body.scrollHeight')

    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(3)
        new_height = driver.execute_script('return document.body.scrollHeight')
        
        if new_height == previous_height:
            break
        previous_height = new_height

    #take the html source
    page_html = driver.page_source
    page_soup = bs4.BeautifulSoup(page_html, 'html.parser')

    #find all content
    content = page_soup.find_all('div', {'class': "tiktok-x6y88p-DivItemContainerV2 e19c29qe7"})
    len_content = len(content)
    # Loop in the profile to find the same URL
    for j in range(1, len_content + 1):
        # Find URL
        try:
            content_str = str(content[j])  # Convert content to a string
            pattern = r'href="(.*?)"'
            match = re.search(pattern, content_str)
            if match:
                get_link_url = match.group(1)
            else:
                links.append(link)
                title.append('skip')
                views.append('skip')
                break

            if get_link_url != link:
                continue

            # Find views
            view_pattern = r'<strong class="video-count[^>]*>(.*?)</strong>'
            view_match = re.search(view_pattern, content_str)
            if view_match:
                get_link_views = view_match.group(1)

            else:
                get_link_views = 'skip'

            links.append(get_link_url)
            views.append(get_link_views)
            # Find title
            title_pattern = r'alt="(.*?)" class="tiktok-1itcwxg-ImgPoster'
            title_match = re.search(title_pattern, content_str)
            if title_match:
                get_link_title = title_match.group(1)
                title.append(get_link_title)
                break
            else:
                get_link_title = 'skip'
            
            if j+3==len_content:
                break

            if j==len_content-3:
                break
        except:
            links.append(link)
            title.append('skip')
            views.append('skip')
            break


    # exit chrome
    driver.quit()
    time.sleep(1)

In [ ]:
# Create a DataFrame
df_result = pd.DataFrame({'Links': links, 'Titles': title, 'Username': username, 'Likes': likes, 'Views': views, 'Comment': comment, 'Share': share})
df_result['insert_date']=str(pd.Timestamp.now())

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

spreadsheetId = "1NuPnWJ2IRPRQ6KzR2jH0bqCA3mTSjBy8F446SbZZRF0"  # Please set the Spreadsheet ID.

scope = ['https://www.googleapis.com/auth/spreadsheets']
credentials = ServiceAccountCredentials.from_json_keyfile_name('D://trial_key.json', scope)
client = gspread.authorize(credentials)
spreadsheet = client.open_by_key(spreadsheetId)
worksheet = spreadsheet.sheet1

data = df_result.values.tolist()

# Update the cell range starting from A1
cell_range = 'B2'
worksheet.update(cell_range, data)